In [44]:
import csv
from datetime import datetime
from datetime import timedelta
import os
import glob
from obspy import read, UTCDateTime, Stream
from obspy.signal.trigger import ar_pick
import matplotlib.pyplot as plt
from tqdm import tqdm

In [45]:
path_data = './AA/202021AAS/'
path_inputfile = './AA/AAS_phasenet_input.csv'
path_parpicks = './AA/AAS_all_arpick_p_picks.csv'
path_sarpicks = './AA/AAS_all_arpick_s_picks.csv'

In [46]:
path_project_home = '/Users/utkuunal/Desktop/Projects/ProjectA'
os.chdir('{}'.format(path_project_home))

In [47]:
picks = []
with open(path_inputfile) as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    for row in csv_reader:
        if row[0] != 'fname':
            dirpart = row[0].split('/')[3]
            filepart = row[0].split('/')[4][:-1]
            picks.append(dirpart + '/' + filepart)
print(len(picks))
print(picks[:10])

7091
['20200514_011723--40.81-25.59--Ke-EGE-DENIZI.M=2.0/RDO.HH', '20200514_011723--40.81-25.59--Ke-EGE-DENIZI.M=2.0/ALN.HH', '20200514_011723--40.81-25.59--Ke-EGE-DENIZI.M=2.0/ENEZ.BH', '20200514_011723--40.81-25.59--Ke-EGE-DENIZI.M=2.0/GADA.BH', '20200514_011723--40.81-25.59--Ke-EGE-DENIZI.M=2.0/ERIK.BH', '20200514_011723--40.81-25.59--Ke-EGE-DENIZI.M=2.0/GELI.BH', '20200514_011723--40.81-25.59--Ke-EGE-DENIZI.M=2.0/KDZ.HH', '20200514_011723--40.81-25.59--Ke-EGE-DENIZI.M=2.0/COMU.HH', '20200514_011723--40.81-25.59--Ke-EGE-DENIZI.M=2.0/BOZC.BH', '20200514_021748--39.59-26.09--Ke-TUZLA-AYVACIK-(CANAKKALE).M=1.3/BOZC.BH']


In [48]:
os.chdir('{}'.format(path_project_home))
i = 0
j = 0
parpicks = []
sarpicks = []
for pick in tqdm(picks):
    current_eq_dir = path_data + pick.split('/')[0] + '/'
    current_station = pick.split('/')[1]
    os.chdir('{}'.format(current_eq_dir))
    current_seismographs = glob.glob('{}*'.format(current_station))
    if current_seismographs != []:
        i += 1
        start_time = UTCDateTime(0)
        end_time = UTCDateTime(2030, 1, 1, 0, 0)
        for k in range(3):
            if current_seismographs[k].split('.')[1][-1] == 'E':
                st_e = read(current_seismographs[k])
                x_e = st_e[0]
                if x_e.stats.sampling_rate != 100:
                    st_e.resample(100)
                    x_e = st_e[0]
                if UTCDateTime(x_e.stats.starttime) > start_time:
                    start_time = UTCDateTime(x_e.stats.starttime)
                if UTCDateTime(x_e.stats.endtime) < end_time:
                    end_time = UTCDateTime(x_e.stats.endtime)
            elif current_seismographs[k].split('.')[1][-1] == 'N':
                st_n = read(current_seismographs[k])
                x_n = st_n[0]
                if x_n.stats.sampling_rate != 100:
                    st_n.resample(100)
                    x_n = st_n[0]
                if UTCDateTime(x_n.stats.starttime) > start_time:
                    start_time = UTCDateTime(x_n.stats.starttime)
                if UTCDateTime(x_n.stats.endtime) < end_time:
                    end_time = UTCDateTime(x_n.stats.endtime)  
            else:
                st_z = read(current_seismographs[k])
                x_z = st_z[0]
                if x_z.stats.sampling_rate != 100:
                    st_z.resample(100)
                    x_z = st_z[0]                
                if UTCDateTime(x_z.stats.starttime) > start_time:
                    start_time = UTCDateTime(x_z.stats.starttime)
                if UTCDateTime(x_z.stats.endtime) < end_time:
                    end_time = UTCDateTime(x_z.stats.endtime)
        if start_time < end_time:
            st_e_new = st_e.slice(start_time, end_time)
            st_n_new = st_n.slice(start_time, end_time)
            st_z_new = st_z.slice(start_time, end_time)
            x_e_new = st_e_new[0]
            x_n_new = st_n_new[0]
            x_z_new = st_z_new[0]
            p_pick, s_pick = ar_pick(
                a=x_z_new.data, b=x_n_new.data, c=x_e_new.data,
                samp_rate=100,
                f1=1, f2=15,
                lta_p=1, sta_p=0.1,
                lta_s=4, sta_s=1,
                m_p=2, m_s=8,
                l_p=0.1, l_s=0.2
            )
            if p_pick < 30:
                parpicks.append([pick, str(x_z_new.stats.starttime + p_pick)[:-5], 'P'])
            if s_pick > 5:
                sarpicks.append([pick, str(x_z_new.stats.starttime + s_pick)[:-5], 'S'])
        else:
            print('ERROR with starttime or endtime ', m, current_eq_dir, current_seismographs)
            print(start_time, end_time)
    else:
        j += 1
    os.chdir('../../..')
print('Picks exist:', i)
print('Picks without seismographs:', j)
print(len(parpicks))
print(parpicks[:10])
print(len(sarpicks))
print(sarpicks[:10])

100%|████████████████████████████████████████████████████████████████████████████████████████████| 7091/7091 [00:33<00:00, 212.31it/s]

Picks exist: 7091
Picks without seismographs: 0
7091
[['20200514_011723--40.81-25.59--Ke-EGE-DENIZI.M=2.0/RDO.HH', '2020-05-14T01:17:30.57', 'P'], ['20200514_011723--40.81-25.59--Ke-EGE-DENIZI.M=2.0/ALN.HH', '2020-05-14T01:17:30.66', 'P'], ['20200514_011723--40.81-25.59--Ke-EGE-DENIZI.M=2.0/ENEZ.BH', '2020-05-14T01:17:32.38', 'P'], ['20200514_011723--40.81-25.59--Ke-EGE-DENIZI.M=2.0/GADA.BH', '2020-05-14T01:17:36.68', 'P'], ['20200514_011723--40.81-25.59--Ke-EGE-DENIZI.M=2.0/ERIK.BH', '2020-05-14T01:17:37.16', 'P'], ['20200514_011723--40.81-25.59--Ke-EGE-DENIZI.M=2.0/GELI.BH', '2020-05-14T01:17:38.76', 'P'], ['20200514_011723--40.81-25.59--Ke-EGE-DENIZI.M=2.0/KDZ.HH', '2020-05-14T01:17:38.61', 'P'], ['20200514_011723--40.81-25.59--Ke-EGE-DENIZI.M=2.0/COMU.HH', '2020-05-14T01:17:42.98', 'P'], ['20200514_011723--40.81-25.59--Ke-EGE-DENIZI.M=2.0/BOZC.BH', '2020-05-14T01:17:43.05', 'P'], ['20200514_021748--39.59-26.09--Ke-TUZLA-AYVACIK-(CANAKKALE).M=1.3/BOZC.BH', '2020-05-14T02:17:54.52', 

In [49]:
i = 0
with open(path_parpicks, 'w', newline='') as file:
    writer = csv.writer(file)
    for parpick in parpicks:
        writer.writerow(parpick)
        i += 1
print(i)

7091


In [50]:
i = 0
with open(path_sarpicks, 'w', newline='') as file:
    writer = csv.writer(file)
    for sarpick in sarpicks:
        writer.writerow(sarpick)
        i += 1
print(i)

6822
